# Introducción

Estos son los datos planificados a guardar y acumular con el objetivo que sirvan para procesos de forecasting eléctrico.

|          Categoría | Widget                                       | Descripción                                                            |
| -----------------: | :------------------------------------------- | :--------------------------------------------------------------------- |
|       **mercados** | `componentes-precio-energia-cierre-desglose` | Desglose horario de los componentes del precio (pool, peajes…)         |
|                    | `energia-precios-ponderados-gestion-desvios` | Precio energético ponderado horario, incluyendo desviaciones           |
|                    | `precios-mercados-tiempo-real`               | Precio intradiario (“real time”) por hora                              |
|                    | `servicio_ajuste`                            | Volúmenes y precios horarios del servicio de ajuste de balance         |
|        **demanda** | `evolucion_demanda`                          | Serie histórica de la demanda eléctrica total por hora                 |
|                    | `ire-general`                                | Consumo eléctrico instantáneo real por hora                            |
|     **generacion** | `evolucion-renovable-no-renovable`           | Evolución horaria de producción renovable vs. no renovable             |
|                    | `estructura-generacion`                      | Mix horario de generación por tecnología (renovable, nuclear, carbón…) |
|                    | `potencia-instalada`                         | Potencia instalada total por tecnología                                |
|   **intercambios** | `todas-fronteras-fisicos`                    | Flujos horarios físicos de intercambio con todas las fronteras         |
|        **balance** | `balance-electrico`                          | Balance horario entre generación y demanda                             |
| **almacenamiento** | `energia-almacenamiento`                     | Energía almacenada por hora en instalaciones de almacenamiento         |
|                    | `potencia-instalada`                         | Capacidad de potencia instalada en instalaciones de almacenamiento     |


Los datos existentes en la API parecen iniciar desde enero de 2014 

Para todos los casos intento priorizar `time_trunc = hour` si existe, para obtener datos horarios mejor segmentados aunque en muchas peticiones ese valor no existe y solo está disponible mensual o anual. En función de como aparece esa tabla o petición disponible, lo he ido modificando. 

Para determinarlo intento ver los datos desde la propia web en la escala temporal mínima que me permita.

# Librerias

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pathlib import Path
import json
from functools import reduce

# Configuración

Al ejecutar esta celda se crean las carpetas donde se irían descargando los archivos para mejor organización, en caso que ya existan lo ignora.

In [2]:
# Ruta base para descargas
base_path = Path('../descargas/ree')

# Definición de categorías y widgets
endpoints = {
    'mercados': [
        'componentes-precio-energia-cierre-desglose',
        'energia-precios-ponderados-gestion-desvios',
        'precios-mercados-tiempo-real',
        'energia-gestionada-servicios-ajuste',
    ],
    'demanda': [
        'evolucion',
        'ire-general',
    ],
    'generacion': [
        'evolucion-renovable-no-renovable',
        'estructura-generacion',
        'potencia-instalada',
    ],
    'intercambios': [
        'todas-fronteras-fisicos',
    ],
    'balance': [
        'balance-electrico',
    ],
    'almacenamiento':[
        'energia-almacenamiento',
        'potencia-instalada',

    ],
}

# Crear carpetas según la estructura definida
for categoria, widgets in endpoints.items():
    for widget in widgets:
        carpeta = base_path / categoria / widget
        if carpeta.exists():
            print(f'Carpeta ya existe: {carpeta}')
        else:
            carpeta.mkdir(parents=True, exist_ok=True)
            print(f'Carpeta creada: {carpeta}')

Carpeta creada: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose
Carpeta creada: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios
Carpeta creada: ..\descargas\ree\mercados\precios-mercados-tiempo-real
Carpeta creada: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste
Carpeta creada: ..\descargas\ree\demanda\evolucion
Carpeta creada: ..\descargas\ree\demanda\ire-general
Carpeta creada: ..\descargas\ree\generacion\evolucion-renovable-no-renovable
Carpeta creada: ..\descargas\ree\generacion\estructura-generacion
Carpeta creada: ..\descargas\ree\generacion\potencia-instalada
Carpeta creada: ..\descargas\ree\intercambios\todas-fronteras-fisicos
Carpeta creada: ..\descargas\ree\balance\balance-electrico
Carpeta creada: ..\descargas\ree\almacenamiento\energia-almacenamiento
Carpeta creada: ..\descargas\ree\almacenamiento\potencia-instalada


# Funciones

Descarga y guarda los archivos json con la respuesta a la petición de la API, con esto se puede visualizar correctamente el json usando por ejemplo extención de Google que facilite ver como se componen las ramas del archivo para generar las tablas que se guardan.

In [ ]:
def download_and_save_json(category, widget, start_date, end_date, time_trunc, base_path):
    """
    Descarga JSON desde la API de REE en tramos temporales y guarda en la ruta adecuada.
    Si time_trunc == 'hour', segmenta mensualmente; en otro caso, segmenta anualmente.

    Args:
        category (str): Categoría en la API.
        widget (str): Nombre del widget/endpoint.
        start_date (str): Fecha de inicio en formato 'YYYY-MM-DDTHH:MM'.
        end_date   (str): Fecha de fin en formato 'YYYY-MM-DDTHH:MM'.
        time_trunc (str): Nivel de agregación temporal ('hour', 'day', etc.).
        base_path  (str o Path): Ruta base donde se guardan las descargas.
    """
    BASE_URL = "https://apidatos.ree.es/es/datos"

    def generate_segments(start, end, trunc):
        dt_start = datetime.strptime(start, '%Y-%m-%dT%H:%M')
        dt_end   = datetime.strptime(end,   '%Y-%m-%dT%H:%M')
        segments = []
        # Elegir intervalo: mensual para hourly, anual para el resto
        delta_arg = {'months': 1} if trunc == 'hour' else {'years': 1}
        while dt_start < dt_end:
            seg_end = dt_start + relativedelta(**delta_arg) - timedelta(minutes=1)
            if seg_end > dt_end:
                seg_end = dt_end
            segments.append((
                dt_start.strftime('%Y-%m-%dT%H:%M'),
                seg_end.strftime('%Y-%m-%dT%H:%M')
            ))
            dt_start = seg_end + timedelta(minutes=1)
        return segments

    # Generar tramos y descargar cada uno
    for seg_start, seg_end in generate_segments(start_date, end_date, time_trunc):
        url = f"{BASE_URL}/{category}/{widget}"
        params = {
            "start_date": seg_start,
            "end_date": seg_end,
            "time_trunc": time_trunc
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Nombre de fichero seguro
        safe_start = seg_start.replace(":", "").replace("T", "_")
        safe_end   = seg_end.replace(":", "").replace("T", "_")
        filename   = f"{widget}_{safe_start}_{safe_end}.json"

        # Carpeta destino
        folder = Path(base_path) / category / widget
        folder.mkdir(parents=True, exist_ok=True)

        # Guardar JSON
        file_path = folder / filename
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"Guardado: {file_path}")


Esta función toma los json dentro de la carpeta correspondiente y crea las tablas según donde se encuentren los valores.

In [35]:
def parse_componentes_json(path, time_trunc):
    """
    Parsea JSON(s) y admite:
    - Caso hourly (attrs.values)
    - Caso monthly estándar (attrs.content)
    - Caso fallback: attrs.values en lugar de attrs.content

    Parsea todos los JSON de una carpeta (o archivo) y devuelve un DataFrame
    donde cada columna es el 'title' de cada serie y los registros son los 'value'
    para cada fecha/hora (formateado a YYYY-MM-DD para monthly, o con hora si hourly).

    Args:
        path (str or Path): Ruta a carpeta o archivo JSON.
        time_trunc (str): Nivel de agregación ('hour' para hourly, else monthly).
    """
    path = Path(path)
    json_files = sorted(path.glob('*.json')) if path.is_dir() else [path]
    dfs_por_archivo = []

    for file_path in json_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            js = json.load(f)

        series_dfs = []
        for group in js.get('included', []):
            attrs = group.get('attributes', {})

            # Caso hourly
            if time_trunc == 'hour' and 'values' in attrs:
                title  = attrs.get('title')
                values = attrs.get('values', [])
                if values:
                    df_temp = pd.DataFrame(values)[['datetime','value']].rename(columns={'value': title})
                    series_dfs.append(df_temp)

            # Monthly u otros
            else:
                # 1) intento contenido estándar
                contents = attrs.get('content', [])
                if contents:
                    for content in contents:
                        c_attr = content.get('attributes', {})
                        title  = c_attr.get('title')
                        values = c_attr.get('values', [])
                        if values:
                            df_temp = pd.DataFrame(values)[['datetime','value']].rename(columns={'value': title})
                            series_dfs.append(df_temp)

                # 2) fallback: 'values' en mismo nivel que 'content'
                elif 'values' in attrs:
                    title  = attrs.get('title')
                    values = attrs.get('values', [])
                    if values:
                        df_temp = pd.DataFrame(values)[['datetime','value']].rename(columns={'value': title})
                        series_dfs.append(df_temp)

        if series_dfs:
            df_merged = reduce(lambda l, r: pd.merge(l, r, on='datetime', how='outer'), series_dfs)
            dfs_por_archivo.append(df_merged)

    if not dfs_por_archivo:
        return pd.DataFrame()

    df_master = pd.concat(dfs_por_archivo, ignore_index=True)
    # Formateo final de 'datetime'
    if time_trunc == 'hour':
        df_master['datetime'] = pd.to_datetime(df_master['datetime'].str[:19])
    else:
        df_master['datetime'] = pd.to_datetime(df_master['datetime'].str[:10])
    return df_master


# Mercados

## componentes-precio-energia-cierre-desglose

### Descarga

En este código usado para todas las secciones, se modifican las fechas y con esto se realiza la descargar del archivo JSON en su carpeta correspondiente. 

Para esta descarga en time_trunc solo admite month o year, aunque anual no tiene sentido.

In [5]:
download_and_save_json(
    'mercados',
    'componentes-precio-energia-cierre-desglose',
    '2014-01-01T00:00',
    '2025-06-19T23:59',
    'month',
    base_path
)

Guardado: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose\componentes-precio-energia-cierre-desglose_2014-01-01_0000_2014-12-31_2359.json
Guardado: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose\componentes-precio-energia-cierre-desglose_2015-01-01_0000_2015-12-31_2359.json
Guardado: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose\componentes-precio-energia-cierre-desglose_2016-01-01_0000_2016-12-31_2359.json
Guardado: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose\componentes-precio-energia-cierre-desglose_2017-01-01_0000_2017-12-31_2359.json
Guardado: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose\componentes-precio-energia-cierre-desglose_2018-01-01_0000_2018-12-31_2359.json
Guardado: ..\descargas\ree\mercados\componentes-precio-energia-cierre-desglose\componentes-precio-energia-cierre-desglose_2019-01-01_0000_2019-12-31_2359.json
Guardado: ..\descargas\ree\mercados\componente

### Procesado

In [36]:
folder = Path('../descargas/ree/mercados/componentes-precio-energia-cierre-desglose')
df_1 = parse_componentes_json(folder, 'month')
df_1.head()

,datetime,Mercado diario,Mercado intradiario (subastas MIBEL y continuo),Restricciones técnicas PDBF,Banda de regulación secundaria,Restricciones técnicas en tiempo real,Incumplimiento de energía de balance,Coste desvíos,Saldo desvíos,Control del factor de potencia,Saldo PO 14.6,Servicios de ajuste,Pagos por capacidad,Precio total (€/MWh),Energía de cierre (MWh),Mecanismo ajuste RD-L 10/2022
0,2014-01-01,36.39,-0.08,4.11,1.44,0.29,NaN,0.48,-0.10,0.0,0.02,6.24,7.00,50.51,2.115419e+07,NaN
1,2014-02-01,18.77,-0.12,3.75,1.42,0.69,NaN,0.22,0.08,0.0,0.01,6.17,6.89,33.33,1.947039e+07,NaN
2,2014-03-01,27.90,-0.07,4.20,0.87,0.58,NaN,0.42,0.01,0.0,0.00,6.08,5.46,40.32,2.026805e+07,NaN
3,2014-04-01,27.26,-0.06,4.99,0.93,1.17,NaN,0.27,0.01,0.0,-0.01,7.36,5.29,40.49,1.826924e+07,NaN
4,2014-05-01,43.18,0.00,3.52,0.81,0.26,NaN,0.20,0.01,0.0,0.01,4.81,5.14,53.20,1.921606e+07,NaN


## energia-precios-ponderados-gestion-desvios

### Descarga

In [7]:
download_and_save_json(
    'mercados',
    'energia-precios-ponderados-gestion-desvios',
    '2014-01-01T00:00',
    '2025-06-19T23:59',
    'month',
    base_path
)


Guardado: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios\energia-precios-ponderados-gestion-desvios_2014-01-01_0000_2014-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios\energia-precios-ponderados-gestion-desvios_2015-01-01_0000_2015-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios\energia-precios-ponderados-gestion-desvios_2016-01-01_0000_2016-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios\energia-precios-ponderados-gestion-desvios_2017-01-01_0000_2017-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios\energia-precios-ponderados-gestion-desvios_2018-01-01_0000_2018-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-precios-ponderados-gestion-desvios\energia-precios-ponderados-gestion-desvios_2019-01-01_0000_2019-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-pr

### Procesado

In [37]:
folder = Path('../descargas/ree/mercados/energia-precios-ponderados-gestion-desvios')
df_2 = parse_componentes_json(folder, 'month')
df_2.head()

,datetime,Precio a subir gestión desvíos,Precio a bajar gestión desvíos,Precio energías de balance RR,Divisor
0,2014-01-01,49.484920,6.501511,7651643.90,225669.9
1,2014-02-01,38.228669,1.215848,6202008.17,235842.5
2,2014-03-01,39.912608,6.348795,5021400.25,240421.0
3,2014-04-01,37.841717,2.620150,4387850.04,191512.4
4,2014-05-01,50.201174,19.997085,5748427.99,142551.5


## precios-mercados-tiempo-real

### Descarga

In [9]:
download_and_save_json(
    'mercados',
    'precios-mercados-tiempo-real',
    '2014-01-01T00:00',
    '2025-06-19T23:59',
    'hour',
    base_path
)


Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-01-01_0000_2014-01-31_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-02-01_0000_2014-02-28_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-03-01_0000_2014-03-31_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-04-01_0000_2014-04-30_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-05-01_0000_2014-05-31_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-06-01_0000_2014-06-30_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-tiempo-real_2014-07-01_0000_2014-07-31_2359.json
Guardado: ..\descargas\ree\mercados\precios-mercados-tiempo-real\precios-mercados-t

### Procesado

In [38]:
folder = Path('../descargas/ree/mercados/precios-mercados-tiempo-real')
df_3 = parse_componentes_json(folder, 'hour')
df_3.head()

,datetime,Precio mercado spot,PVPC
0,2014-01-01 00:00:00,20.02,NaN
1,2014-01-01 01:00:00,10.34,NaN
2,2014-01-01 02:00:00,5.35,NaN
3,2014-01-01 03:00:00,5.00,NaN
4,2014-01-01 04:00:00,0.50,NaN


## energia-gestionada-servicios-ajuste

### Descarga

In [16]:
download_and_save_json(
    'mercados',
    'energia-gestionada-servicios-ajuste',
    '2014-01-01T00:00',
    '2025-06-19T23:59',
    'month',
    base_path
)


Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2014-01-01_0000_2014-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2015-01-01_0000_2015-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2016-01-01_0000_2016-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2017-01-01_0000_2017-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2018-01-01_0000_2018-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2019-01-01_0000_2019-12-31_2359.json
Guardado: ..\descargas\ree\mercados\energia-gestionada-servicios-ajuste\energia-gestionada-servicios-ajuste_2020-01-01_0000_2020-1

### Procesado

In [39]:
folder = Path('../descargas/ree/mercados/energia-gestionada-servicios-ajuste')
df_4 = parse_componentes_json(folder,'month')
df_4.head()

,datetime,Energía programada por seguridad a subir,Energía programada por seguridad a bajar,Energía de balance a subir,Energía de balance a bajar
0,2014-01-01,1052529.9,-233744.6,592420.1,-355635.4
1,2014-02-01,952533.6,-336101.1,512839.9,-339301.1
2,2014-03-01,1113618.3,-200928.5,452573.1,-476333.0
3,2014-04-01,1161613.3,-65465.4,531782.7,-327268.1
4,2014-05-01,940816.1,-86008.4,496273.9,-285274.6


Para el resto de categorías es repetir el mismo patrón y si corresponde alguna ligera modificación de las dos funciones principales para tomar los valores correctos de los json descargados

Al finalizar según corresponda guardar los df por separados o unificar aquellos que la serie temporal lo permita.